In [1]:
import pandas as pd
import numpy as np
from scipy import interpolate

# Load data
timeseries_df = pd.read_csv('files/timeseries_la_haute_borne_2017.csv')
power_curve_df = pd.read_csv('files/Senvion_MM82.csv')

# Constants
rho = 1.225  # density [kg/m^3]
D = 82  # rotor_diameter [m]
T = 8760  # total hours/years [h]
A = np.pi * (D / 2) ** 2 # swept area [m^2]
cut_in_ws = power_curve_df[power_curve_df['P'] > 0]['ws'].min()
cut_out_ws = power_curve_df[power_curve_df['P'] > 0]['ws'].max()

# Clear NaN Values & Calcualte WS
timeseries_df.dropna(subset=['U', 'V'], inplace=True) # Drop rows where specific columns have NaN values. 'Inplace=True' => modify the data frame directly and'Inplace=False' will return a new data frame.
timeseries_df['WS'] = np.sqrt(timeseries_df['U']**2 + timeseries_df['V']**2)

# Interpolate cp using the power curve
cp_i = interpolate.interp1d(power_curve_df['ws'], power_curve_df['cp'])

# Function (later more function to be added for other tasks)
def calculate_APP_entire_timeseries(timeseries_df, cp_i):
    
    valid_ws = timeseries_df['WS'].between(cut_in_ws, cut_out_ws, inclusive='both')
    cp = cp_i(timeseries_df['WS'][valid_ws])
    delta_t = 10 / 60  # 10 min in 1h
    power = 0.5 * rho * A * cp * timeseries_df['WS'][valid_ws]**3 * delta_t
    APP = power.sum() / T

    print(f"The APP based on the entired timeseries is: {APP/1000:.2f} kW")

    return APP

#Execution
APP_ET = calculate_APP_entire_timeseries(timeseries_df, cp_i)


The APP based on the entired timeseries is: 314.67 kW
